# BCHO PCS

In [1]:
from group import DummyGroup
from utils import log_2, pow_2
from unipolynomial import UniPolynomial
from mle2 import MLEPolynomial
from kzg10 import Commitment, KZG10Commitment

In [2]:
F193.<a> = GF(193)
R193.<X> = F193[]
R193
Fp=F193
Fp

Finite Field of size 193

In [3]:
UniPolynomial.set_scalar(Fp(0), Fp)
Commitment.set_scalar(Fp(0))

In [4]:
# For symbolic calculation

R_2.<X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2> = Fp[]; R_2

Multivariate Polynomial Ring in X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2 over Finite Field of size 193

In [5]:
Fp2 = Fp.extension(2, 'b'); Fp2

Finite Field in b of size 193^2

In [6]:
omega = Fp.primitive_element()^3; omega, omega^64 == 1, omega^32 == -1

(125, True, True)

In [7]:
H16 = [(omega^4)^i for i in range(16)]; H16

[1, 64, 43, 50, 112, 27, 184, 3, 192, 129, 150, 143, 81, 166, 9, 190]

In [8]:
H8 = [(omega^8)^i for i in range(8)]; H8

[1, 43, 112, 184, 192, 150, 81, 9]

In [9]:
H4 = [(omega^16)^i for i in range(4)]; H4

[1, 112, 192, 81]

In [10]:
G1 = DummyGroup(Fp)
G1.generator(), G1.exp(2, 3), G1.order(), 

(5, 6, 193)

In [11]:
G1.exp(X1, 3)

3*X1

In [12]:
G2 = DummyGroup(Fp2)
Fp2.random_element()
G2.generator(), G2.exp(G2.generator(), 3), G2.order(), G2.field.polynomial()

(b, 3*b, 37249, b^2 + 192*b + 5)

In [13]:
def is_power_of_two(n):
    d = n
    k = 0
    while d > 0:
        d >>= 1
        k += 1
    if n == 2**(k-1):
        return True
    else:
        return False

def next_power_of_two(n):
    assert n >= 0, "No negative integer"
    if is_power_of_two(n):
        return n
    d = n
    k = 0
    while d > 0:
        d >>= 1
        k += 1
    return k

is_power_of_two(15), next_power_of_two(15)

(False, 4)

In [14]:
def bits_le_with_width(i, width):
    if i >= 2**width:
        return "Failed"
    bits = []
    while width:
        bits.append(i % 2)
        i //= 2
        width -= 1
    return bits
        
bits_le_with_width(6, 5)


[0, 1, 1, 0, 0]

## Polynomial

In [15]:
# 
#  m = [1]
#  m = [1] + [1 * X0, 1 * X1]
#  m = m + [1 * X0^2, 1 * X0 * X1, 1 * X1]
def mle_eval_from_coeffs(f_coeffs, point):
    k = len(point)
    n = len(f_coeffs)
    assert len(f_coeffs) == 1 << k
    
    e = Fp(0)
    for i in range(n):
        i_bits = bits_le_with_width(i, k)
        monomial = product([point[j] if i_bits[j] == 1 else 1 for j in range(k)])
        e += f_coeffs[i] * monomial
    
    return e

mle_eval_from_coeffs([1, 3, 2, 1], [91, 181]) == Fp((1 + 3*X0 + 2*X1 + 1*X0*X1)(X0=91, X1=181))

True

In [16]:
mle_eval_from_coeffs([1, 3, 2, 1], [X0, X1])

X0*X1 + 3*X0 + 2*X1 + 1

In [17]:
def uni_eval_from_coeffs(coeffs, z):
    t = Fp(1)
    eval = Fp(0)
    for i in range(0, len(coeffs)):
        eval += coeffs[i] * t
        t *= z
    return eval

uni_eval_from_coeffs([1, 3, 2, 1], 91) == Fp(1 + 3*91 + 2*91^2 + 1*91^3) == Fp(144)

True

In [18]:
def polynomial_division(dividend, divisor):
    """
    Perform univariate polynomial division.

    Args:
        dividend (list): Coefficients of the dividend polynomial, in descending order of degree.
        divisor (list): Coefficients of the divisor polynomial, in descending order of degree.

    Returns:
        tuple: (quotient, remainder), where quotient and remainder are lists of coefficients.

    Raises:
        ValueError: If the divisor is zero or of higher degree than the dividend.
    """
    # Remove leading zeros
    while dividend and dividend[0] == 0:
        dividend = dividend[1:]
    while divisor and divisor[0] == 0:
        divisor = divisor[1:]

    # Check for zero division
    if not divisor:
        raise ValueError("Division by zero polynomial")

    # Check degrees
    if len(dividend) < len(divisor):
        return [], dividend

    # Initialize quotient and remainder
    quotient = [0] * (len(dividend) - len(divisor) + 1)
    remainder = dividend.copy()

    # Perform polynomial long division
    for i in range(len(quotient)):
        if len(remainder) < len(divisor):
            break
        
        # Compute the next term of the quotient
        term = remainder[0] // divisor[0]
        quotient[i] = term

        # Subtract term * divisor from the remainder
        for j in range(len(divisor)):
            if j < len(remainder):
                remainder[j] -= term * divisor[j]

        # Remove leading zero from remainder
        while remainder and remainder[0] == 0:
            remainder.pop(0)

    # Remove trailing zeros from quotient
    while quotient and quotient[-1] == 0:
        quotient.pop()

    return quotient, remainder

In [19]:
# Example usage
dividend = [Fp(1), Fp(0), Fp(-2), Fp(0), Fp(1)]  # x^4 - 2x^2 + 1
divisor = [Fp(1), Fp(1)]  # x + 1

quotient, remainder = polynomial_division(dividend, divisor)

print("Dividend:", dividend)
print("Divisor:", divisor)
print("Quotient:", quotient)
print("Remainder:", remainder)

Dividend: [1, 0, 191, 0, 1]
Divisor: [1, 1]
Quotient: [1, 192, 192, 1]
Remainder: []


In [20]:
(X^4 - 2*X^2 + 1)/(X+1)

X^3 + 192*X^2 + 192*X + 1

In [21]:
kzg10 = KZG10Commitment(G1, G2, 10); 

In [22]:
kzg10.setup(secret_symbol=X0, g1_generator=1, g2_generator=1)
kzg10.srs


[1, X0, X0^2, X0^3, X0^4, X0^5, X0^6, X0^7, X0^8, X0^9, X0^10]

In [23]:
kzg10.h, kzg10.h_s

(1, X0)

In [24]:
kzg10.commit([3,2,1]), kzg10.commit([3,2,1])

(Commitment(X0^2 + 2*X0 + 3), Commitment(X0^2 + 2*X0 + 3))

In [25]:
C0 = kzg10.commit([3,2,1]); C0

Commitment(X0^2 + 2*X0 + 3)

In [26]:
C1 = kzg10.commit([9,10,11]); C1

Commitment(11*X0^2 + 10*X0 + 9)

In [27]:
C0 + C1

Commitment(12*X0^2 + 12*X0 + 12)

In [28]:
W = kzg10.prove_eval([3,2,1], 91, 167)

In [29]:
kzg10.verify_eval(C0, W, 91, 167)


True

In [30]:
UniPolynomial([93,1]) * UniPolynomial([-91, 1]) + UniPolynomial([167])

-8296 + 2x + x^2

In [31]:
def log_2(x):
    """
    Compute the integer part of the logarithm base 2 of x.

    Args:
        x (int): The number to compute the logarithm of. Must be a positive integer.

    Returns:
        int: The floor of the logarithm base 2 of x.

    Raises:
        ValueError: If x is not a positive integer.
    """
    if not isinstance(x, int) or x <= 0:
        raise ValueError("x must be a positive integer")
    
    result = 0
    while x > 1:
        x >>= 1  # Bit shift right (equivalent to integer division by 2)
        result += 1
    return result

## BCHO PCS


In [32]:
class ProofTranscript:
    def __init__(self, field):
        self.field = field
        self.state = 0  # Initial state
    
    def absorb(self, element):
        """
        Absorb a field element into the transcript.
        
        Args:
            element: A field element to be absorbed. 
        """
        if isinstance(element, int):
            element = self.field(element)
        elif isinstance(element, sage.rings.integer.Integer):
            element = self.field(element)
        elif isinstance(element, Commitment):
            element = element.value
        elif isinstance(element, list):
            element = sum(element)
        elif not element.parent() == self.field:
            raise TypeError("Must absorb a field element")
        
        # if not isinstance(element, self.field.Element):
        #     raise TypeError("Must absorb a field element")
        
        # Simple hash function: multiply by a prime and add the element
        bs = str(element).encode('utf-8')
        n = int.from_bytes(bs, 'big')
        self.state = (self.state * int(31) + n) % self.field.order()
    
    def squeeze(self):
        """
        Generate a random field element based on the current state.
        
        Returns:
            A field element.
        """
        # Use the current state to generate a "random" field element
        self.state = (self.state * 31 + 17) % self.field.order()
        return self.field(self.state)
    
    def clone(self):
        new = ProofTranscript(self.field)
        new.state = self.state
        return new


In [33]:
Fp(1).parent() == Fp

True

In [34]:
type(30)

<class 'sage.rings.integer.Integer'>

In [35]:
    
transcript = ProofTranscript(Fp)

# Absorb some elements
transcript.absorb(10)
transcript.absorb(20)

# Squeeze some randomness
r1 = transcript.squeeze()
r2 = transcript.squeeze()

print(f"Random element 1: {r1}")
print(f"Random element 2: {r2}")

# Absorb more elements
transcript.absorb(30)

# Squeeze more randomness
r3 = transcript.squeeze()

print(f"Random element 3: {r3}")

Random element 1: 151
Random element 2: 66
Random element 3: 98


In [36]:
def prove_eval_arg_naive(C, f, point, v, transcript):
    """
    C: commitment to f
    f: polynomial
    point: evaluation point
    v: evaluation value
    Rng: randomness
    """
    
    n = len(f)
    k = log_2(n)
    if len(point) != k:
        raise ValueError("Invalid evaluation point, k={}, while len(point)={}".format(k, len(point)))
    if not is_power_of_two(n):
        raise ValueError("Invalid polynomial length")
    
    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    # Split and fold polynomials
    coeffs = f[:]
    h_uni_cm_vec = []
    h_uni_poly_vec = [coeffs]
    for i in range(k):
        f_even = coeffs[::2]
        f_odd = coeffs[1::2]
        coeffs = [Fp(f_even[j] + point[i] * f_odd[j])  for j in range(len(f_even))]
        h_cm = kzg10.commit(coeffs)

        transcript.absorb(h_cm)

        h_uni_poly_vec.append(coeffs)
        h_uni_cm_vec.append(h_cm)

    print("h_uni_cm_vec=", h_uni_cm_vec)
    print("h_uni_poly_vec=", h_uni_poly_vec)
    assert coeffs == [Fp(v)]
    assert len(coeffs) == 1
    print("coeffs=", coeffs)

    # Sample randomness and reply evaluations
    r = transcript.squeeze()
    print("r=", r)
    evals_pos = []
    evals_neg = []
    evals_sq = []
    for i in range(k):
        poly = h_uni_poly_vec[i]
        evals_pos.append(uni_eval_from_coeffs(poly, r))
        evals_neg.append(uni_eval_from_coeffs(poly, (-1) * r))
        evals_sq.append(uni_eval_from_coeffs(poly, r^2))
    
    
    # DEBUG: verify evaluations
    dbg_evals_sq = evals_sq[:] + [v]
    for i in range(k):
        print("i={}, {}".format(i, dbg_evals_sq[i+1] == (evals_pos[i] + evals_neg[i])/2 + point[i] * (evals_pos[i] - evals_neg[i])/(2*r)))

    return (h_uni_cm_vec, evals_pos, evals_neg, evals_sq)

In [37]:
def verify_eval_arg_naive(C, arg, point, v, transcript):
    """
    C: commitment to f
    arg: argument to be verified
    point: evaluation point
    v: evaluation value
    transcript: proof transcript
    """
    k = len(point)
    n = 1 << k
    
    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    h_uni_cm_vec, evals_pos, evals_neg, evals_sq = arg

    for i in range(len(h_uni_cm_vec)):
        transcript.absorb(h_uni_cm_vec[i])
    
    r = transcript.squeeze()
    
    evals_sq.append(v)
    verified = True
    # Verify evaluations
    for i in range(k):
        b = evals_sq[i+1] == (evals_pos[i] + evals_neg[i])/2 + point[i] * (evals_pos[i] - evals_neg[i])/(2*r)
        print("i={}, {}".format(i, b))
        verified = verified and b
    if verified:
        print("✅ Verified!")
    return verified


In [38]:
transcript = ProofTranscript(Fp); transcript.state

0

In [39]:
ff = [1, 3, 2, 1]
ff_cm = kzg10.commit(ff)
ff_cm

Commitment(X0^3 + 2*X0^2 + 3*X0 + 1)

In [40]:
arg = prove_eval_arg_naive(ff_cm, [1, 3, 2, 1], point=[91, 181], v=123, transcript=transcript.clone())

h_uni_cm_vec= [Commitment(93*X0 + 81), Commitment(-70)]
h_uni_poly_vec= [[1, 3, 2, 1], [81, 93], [123]]
coeffs= [123]
r= 174
i=0, True
i=1, True


In [41]:
uni_eval_from_coeffs([81, 93], 150)

135

In [42]:
verify_eval_arg_naive(ff_cm, arg, point=[91, 181], v=123, transcript=ProofTranscript(Fp))

i=0, True
i=1, True
✅ Verified!


True

## Optimized version

In [43]:
def prove_eval_arg_opt(C, f, point, v, transcript):
    """
    C: commitment to f
    f: polynomial
    point: evaluation point
    v: evaluation value
    Rng: randomness
    """
    
    n = len(f)
    k = log_2(n)
    if len(point) != k:
        raise ValueError("Invalid evaluation point, k={}, while len(point)={}".format(k, len(point)))
    if not is_power_of_two(n):
        raise ValueError("Invalid polynomial length")
    
    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    # Split and fold polynomials
    coeffs = f[:]
    h_uni_cm_vec = []
    h_uni_poly_vec = [coeffs]
    for i in range(k-1):
        f_even = coeffs[::2]
        f_odd = coeffs[1::2]
        coeffs = [Fp(f_even[j] + point[i] * f_odd[j])  for j in range(len(f_even))]
        h_cm = kzg10.commit(coeffs)

        transcript.absorb(h_cm)

        h_uni_poly_vec.append(coeffs)
        h_uni_cm_vec.append(h_cm)

    print("h_uni_cm_vec=", h_uni_cm_vec)
    print("h_uni_poly_vec=", h_uni_poly_vec)
    assert Fp(v) == coeffs[0] + point[k-1] * coeffs[1]
    assert len(coeffs) == 2
    print("coeffs=", coeffs)

    # Sample randomness and reply evaluations
    r = transcript.squeeze()
    print("r=", r)
    evals_pos = []
    evals_neg = []
    evals_sq = []
    for i in range(k):
        poly = h_uni_poly_vec[i]
        poly_at_r = uni_eval_from_coeffs(poly, r)
        poly_at_neg_r = uni_eval_from_coeffs(poly, (-1) * r)
        poly_at_r_sq = uni_eval_from_coeffs(poly, r^2)
        evals_pos.append(poly_at_r)
        evals_neg.append(poly_at_neg_r)
        evals_sq.append(poly_at_r_sq)
        transcript.absorb(poly_at_r)
        transcript.absorb(poly_at_neg_r)
        transcript.absorb(poly_at_r_sq)
    
    print("evals_pos=", evals_pos)
    print("evals_neg=", evals_neg)
    print("evals_sq=", evals_sq)
    
    gamma = transcript.squeeze()
    # gamma = 1
    print("gamma=", gamma)

    h_agg = [0] * n
    for i in range(n):
        for j in range(k):
            poly = h_uni_poly_vec[j]
            if i < len(poly):
                h_agg[i] += poly[i] * (gamma**j)
    print("h_agg=", h_agg)
    
    h_agg_poly_at_r = uni_eval_from_coeffs(h_agg, r)
    h_agg_poly_at_neg_r = uni_eval_from_coeffs(h_agg, -r)
    h_agg_poly_at_r_sq = uni_eval_from_coeffs(h_agg, r^2)

    print("h_agg_poly_at_r={}, h_agg_poly_at_neg_r={}, h_agg_poly_at_r_sq={}".format(h_agg_poly_at_r, h_agg_poly_at_neg_r, h_agg_poly_at_r_sq))

    # DEBUG: verify h_agg
    assert h_agg_poly_at_r == sum([evals_pos[i]*(gamma**i) for i in range(k)])


    # DEBUG: verify evaluations
    dbg_evals_sq = evals_sq[:] + [v]
    for i in range(k):
        print("dbg_evals_sq[i+1]={}, interp_and_eval_line(evals_pos[i], evals_neg[i], r, point[i])={}".format(dbg_evals_sq[i+1], (evals_pos[i] + evals_neg[i])/2 + point[i] * (evals_pos[i] - evals_neg[i])/(2*r)))
        print("i={}, {}".format(i, dbg_evals_sq[i+1] == (evals_pos[i] + evals_neg[i])/2 + point[i] * (evals_pos[i] - evals_neg[i])/(2*r)))

    interp_poly = UniPolynomial.interpolate([h_agg_poly_at_r, h_agg_poly_at_neg_r, h_agg_poly_at_r_sq], [r, -r, r^2])
    print("interp_poly=", interp_poly)
    vanishing_poly = UniPolynomial([-r, 1])*UniPolynomial([r, 1]) * UniPolynomial([-r*r, 1])
    print("vanishing_poly=", vanishing_poly)

    h_agg_poly = UniPolynomial(h_agg)

    quo_poly, rem = divmod(h_agg_poly-interp_poly, vanishing_poly)
    print("quo={}, rem={}".format(quo_poly, rem))
    # assert rem == UniPolynomial([0])
    print("quo_poly={}".format(quo_poly))
    
    g_poly = h_agg_poly - interp_poly
    print("g_poly=", g_poly.coeffs)
    print("rem={}".format(g_poly % vanishing_poly))
    print("quo={}".format(g_poly // vanishing_poly))
    
    Cq = kzg10.commit(quo_poly.coeffs)
    transcript.absorb(Cq)

    zeta = transcript.squeeze()
    print("zeta={}".format(zeta))
    
    interp_at_zeta =interp_poly.evaluate(zeta)
    print("interp_at_zeta={}".format(interp_at_zeta))
    
    r_poly = h_agg_poly - UniPolynomial([interp_at_zeta]) - quo_poly * ((zeta - r) * (zeta + r) * (zeta - r*r))
    print("h_agg_poly - UniPolynomial([interp_at_zeta])=", h_agg_poly - UniPolynomial([interp_at_zeta]))
    print("quo_poly * ((zeta - r) * (zeta + r) * (zeta - r*r))=", quo_poly * ((zeta - r) * (zeta + r) * (zeta - r*r)))
    print("r_poly=", r_poly)

    print("r(zeta)={}".format(r_poly.evaluate(zeta)))

    Cr = kzg10.commit(r_poly.coeffs)

    transcript.absorb(Cr)

    arg_r_poly_at_zeta =kzg10.prove_eval(r_poly.coeffs, point=zeta, v=0)

    # DEBUG: verify r_poly argument
    assert kzg10.verify_eval(Cr, arg_r_poly_at_zeta, point=zeta, v=0)

    return (h_uni_cm_vec, evals_pos, evals_neg, evals_sq, Cq, arg_r_poly_at_zeta)

In [44]:
def verify_eval_arg_opt(C, arg, point, v, transcript):
    """
    Verify an optimized evaluation argument proof.

    Args:
    C: commitment to f
    arg: argument to be verified (output of prove_eval_arg_opt)
    point: evaluation point
    v: claimed evaluation value
    transcript: proof transcript
    """
    k = len(point)
    n = 1 << k
    
    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    print("C=", C)

    print("arg=", arg)

    h_uni_cm_vec, evals_pos, evals_neg, evals_sq, Cq, Cw = arg

    for i in range(len(h_uni_cm_vec)):
        transcript.absorb(h_uni_cm_vec[i])
    
    beta = transcript.squeeze()
    print("beta=", beta)

    for i in range(k):
        transcript.absorb(evals_pos[i])
        transcript.absorb(evals_neg[i])
        transcript.absorb(evals_sq[i])
    
    gamma = transcript.squeeze()
    print("gamma=", gamma)

    Ch = C
    for i in range(0, k-1):
        Ch = Ch + (gamma**(i+1)) * h_uni_cm_vec[i]
    print("Ch=", Ch)
    
    h_agg_poly_at_beta = sum([evals_pos[i]*(gamma**i) for i in range(k)])
    h_agg_poly_at_neg_beta = sum([evals_neg[i]*(gamma**i) for i in range(k)])
    h_agg_poly_at_beta_sq = sum([evals_sq[i]*(gamma**i) for i in range(k)])
    print("h_agg_poly_at_beta={}, h_agg_poly_at_neg_beta={}, h_agg_poly_at_beta_sq={}".format(h_agg_poly_at_beta, h_agg_poly_at_neg_beta, h_agg_poly_at_beta_sq))

    # vanishing_poly = UniPolynomial([-beta, h_agg_poly_at_beta]) * UniPolynomial([beta, h_agg_poly_at_neg_beta]) * UniPolynomial([-beta*beta, h_agg_poly_at_beta_sq])
    interp_poly = UniPolynomial.interpolate(
        [h_agg_poly_at_beta, h_agg_poly_at_neg_beta, h_agg_poly_at_beta_sq], 
        [beta, -beta, beta**2])

    transcript.absorb(Cq)

    zeta = transcript.squeeze()
    print("zeta={}".format(zeta))
    
    # vanishing_poly_at_zeta= vanishing_poly.evaluate(zeta)
    interp_at_zeta = interp_poly.evaluate(zeta)
    print("interp_at_zeta={}".format(interp_at_zeta))

    Cr = Ch - kzg10.commit([interp_at_zeta]) - Cq * ((zeta - beta) * (zeta + beta) * (zeta - beta*beta))

    print("Cr=", Cr)
    print("Cq=", Cq)

    transcript.absorb(Cr)

    # Verify r_poly argument
    if not kzg10.verify_eval(Cr, Cw, point=zeta, v=0):
        return False

    # Verify evaluations
    evals_sq = evals_sq + [v]
    for i in range(k):
        if evals_sq[i+1] != (evals_pos[i] + evals_neg[i])/2 + point[i] * (evals_pos[i] - evals_neg[i])/(2*beta):
            return False

    return True

In [45]:
UniPolynomial.polynomial_division_with_remainder([46, 25, 25, 1], [123, 128, 170])

([126, 151], [181, 64])

In [46]:
UniPolynomial([126, 151]) * UniPolynomial([123, 128, 170]) + UniPolynomial([181, 64])

15679 + 34765x + 40748x^2 + 25670x^3

In [47]:
divmod( UniPolynomial([46, 25, 25, 1]),  UniPolynomial([123, 128, 170]))

(126 + 151x, 181 + 64x)

In [48]:
UniPolynomial([46, 25, 25, 1]) % UniPolynomial([123, 128, 170])

181 + 64x

In [49]:
type(ff_cm)

<class 'kzg10.Commitment'>

In [50]:
arg = prove_eval_arg_opt(ff_cm, [1, 3, 2, 1], point=[91, 181], v=123, transcript=transcript.clone()); arg

h_uni_cm_vec= [Commitment(93*X0 + 81)]
h_uni_poly_vec= [[1, 3, 2, 1], [81, 93]]
coeffs= [81, 93]
r= 177
evals_pos= [36, 137]
evals_neg= [25, 25]
evals_sq= [134, 150]
gamma= 107
h_agg= [176, 111, 2, 1]
h_agg_poly_at_r=27, h_agg_poly_at_neg_r=191, h_agg_poly_at_r_sq=165
dbg_evals_sq[i+1]=150, interp_and_eval_line(evals_pos[i], evals_neg[i], r, point[i])=150
i=0, True
dbg_evals_sq[i+1]=123, interp_and_eval_line(evals_pos[i], evals_neg[i], r, point[i])=123
i=1, True
interp_poly= 67 + 174x + 65x^2
vanishing_poly= 109 + 130x + 130x^2 + x^3
quo=1, rem=0
quo_poly=1
g_poly= [109, 130, 130, 1]
rem=0
quo=1
zeta=143
interp_at_zeta=46
h_agg_poly - UniPolynomial([interp_at_zeta])= 130 + 111x + 2x^2 + x^3
quo_poly * ((zeta - r) * (zeta + r) * (zeta - r*r))= 30
r_poly= 100 + 111x + 2x^2 + x^3
r(zeta)=0


([Commitment(93*X0 + 81)],
 [36, 137],
 [25, 25],
 [134, 150],
 Commitment(1),
 Commitment(X0^2 - 48*X0 + 2))

In [51]:
verify_eval_arg_opt(ff_cm, arg, point=[91, 181], v=123, transcript=transcript.clone())

C= Commitment(X0^3 + 2*X0^2 + 3*X0 + 1)
arg= ([Commitment(93*X0 + 81)], [36, 137], [25, 25], [134, 150], Commitment(1), Commitment(X0^2 - 48*X0 + 2))
beta= 177
gamma= 107
Ch= Commitment(X0^3 + 2*X0^2 - 82*X0 - 17)
h_agg_poly_at_beta=27, h_agg_poly_at_neg_beta=191, h_agg_poly_at_beta_sq=165
zeta=143
interp_at_zeta=46
Cr= Commitment(X0^3 + 2*X0^2 - 82*X0 - 93)
Cq= Commitment(1)


True

In [52]:
gamma=50
(X0^3 + 2*X0^2 + 3*X0 + 1) + (93*X0 + 81) * gamma 

X0^3 + 2*X0^2 + 21*X0 - 2

In [53]:
uni_eval_from_coeffs([1, 3, 2, 1], 174), uni_eval_from_coeffs([81, 93], 174)

(176, 51)

In [54]:
type(transcript.squeeze())

<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>

In [55]:
f1 = UniPolynomial([1, 2, 3])
f2 = UniPolynomial([4, 5, 6])
f3 = f1 + f2
f3

5 + 7x + 9x^2

In [56]:
f1 = UniPolynomial([1, 2, 3])
f2 = UniPolynomial([4, 5, 6])
f3 = f1 * f2
f3

4 + 13x + 28x^2 + 27x^3 + 18x^4

In [57]:
def poly(cof):
    return sum([cof[i] * X^i for i in range(len(cof))])

poly([1, 2, 3])


3*X^2 + 2*X + 1

In [58]:
poly([1, 2, 3]) * poly([4, 5, 6])

18*X^4 + 27*X^3 + 28*X^2 + 13*X + 4

In [59]:
f1 = UniPolynomial([1, 2, 3])
f2 = UniPolynomial([4, 5])
divmod(f1, f2)

(108 + 155x, 148)

In [60]:
(155*X + 108) * poly([4, 5]) + poly([148])

3*X^2 + 2*X + 1

In [61]:
z4 = UniPolynomial.vanishing_polynomial([Fp(1), Fp(2), Fp(3), Fp(4)]); z4

24 + 143x + 35x^2 + 183x^3 + x^4

In [62]:
z4 // (UniPolynomial([-1, 1])), z4 % (UniPolynomial([-1, 1]))

(169 + 26x + 184x^2 + x^3, 0)

In [63]:
z4.division_by_linear_divisor(Fp(1))

(169 + 26x + 184x^2 + x^3, 0)

In [64]:
divmod(z4, UniPolynomial([-1, 1]))

(169 + 26x + 184x^2 + x^3, 0)

In [65]:
UniPolynomial.polynomial_division_with_remainder([24, 143, 35, 183, 1], [Fp(-1), Fp(1)])

([169, 26, 184, 1], [])

In [66]:
divmod(z4, UniPolynomial([-1, 1]))


(169 + 26x + 184x^2 + x^3, 0)

In [67]:
(X-1)*(X-2)*(X-3)*(X-4)

X^4 + 183*X^3 + 35*X^2 + 143*X + 24

In [68]:
cof = UniPolynomial.compute_coeffs_from_evals_fast([5, 3, 2, 4], [1, 2, 3, 4]); cof

[6, 161, 95, 129]

In [69]:
[poly(cof).subs(X=i+1) for i in range(4)]

[5, 3, 2, 4]

In [70]:
UniPolynomial.compute_evals_from_coeffs_fast(cof, [1, 2, 3, 4])

[5, 3, 2, 4]

In [71]:
cof = UniPolynomial.ntt_coeffs_from_evals([5, 3, 2, 4], 2, H4[1]); cof

[100, 77, 0, 21]

In [72]:
UniPolynomial.ntt_evals_from_coeffs(cof, 2, H4[1])

[5, 3, 2, 4]

## Optimization 

1. Prover doesn't need to send $h^{(i)}(\beta^2)$

In [73]:
## Interpolate two points into a polynomial and evaluate it at X=r
#
#   f(X) = [(-x, a), (x, b)] -> (a+b)/2 + ((a-b) / (2*x)) * X
#
#   f(r) = (a+b)/2 + r * (a-b) / (2*x)
#
# 
#(evals_pos[i] + evals_neg[i])/2 + point[i] * (evals_pos[i] - evals_neg[i])/(2*r)))


def interp_and_eval_line(a, b, r, x):
    """
    Interpolate two points into a polynomial and evaluate it at X=r

        f(X) = [(-x, a), (x, b)] -> (a+b)/2 + ((a-b) / (2*x)) * X

        f(r) = (a+b)/2 + r * (a-b) / (2*x)
    Args:
        a (int): The evaluation at -x
        b (int): The evaluation at x
        r (int): The point at which to evaluate the polynomial
        x (int): The x-coordinate of the point

    Returns:
        int: The result of evaluating the polynomial at X=r
    """
    return ((a + b) / 2 + r * (a - b) / (2 * x))

In [74]:
def prove_eval_arg_opt_opt(C, f, point, v, transcript, debug=0):
    """
    C: commitment to f
    f: polynomial
    point: evaluation point
    v: evaluation value
    Rng: randomness
    """
    
    n = len(f)
    k = log_2(n)
    if len(point) != k:
        raise ValueError("Invalid evaluation point, k={}, while len(point)={}".format(k, len(point)))
    if not is_power_of_two(n):
        raise ValueError("Invalid polynomial length")
    
    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    # Split and fold polynomials
    coeffs = f[:]
    h_uni_cm_vec = []
    h_uni_poly_vec = [coeffs]
    for i in range(k-1):
        f_even = coeffs[::2]
        f_odd = coeffs[1::2]
        coeffs = [Fp(f_even[j] + point[i] * f_odd[j])  for j in range(len(f_even))]
        h_cm = kzg10.commit(coeffs)

        transcript.absorb(h_cm)

        h_uni_poly_vec.append(coeffs)
        h_uni_cm_vec.append(h_cm)

    if debug > 0: 
        coeffs = [coeffs[i] + point[k-1] * coeffs[i+1] for i in range(0, len(coeffs), 2)]
        print("check: fully folded polynomial must be equal to the claimed evaluation: {} = {}".format(coeffs[0], v))
        assert (coeffs[0] == v)
        assert len(coeffs) == 1
        print("coeffs=", coeffs)
        print("h_uni_cm_vec=", h_uni_cm_vec)
        print("h_uni_poly_vec=", h_uni_poly_vec)

    # Sample randomness and reply evaluations
    beta = transcript.squeeze()
    print("beta=", beta)
    evals_pos = []
    evals_neg = []    
    evals_sq = []
    for i in range(k):
        poly = h_uni_poly_vec[i]
        poly_at_beta = uni_eval_from_coeffs(poly, beta)
        poly_at_neg_beta = uni_eval_from_coeffs(poly, (-1) * beta)
        poly_at_beta_sq = uni_eval_from_coeffs(poly, beta^2)
        evals_pos.append(poly_at_beta)
        evals_neg.append(poly_at_neg_beta)
        evals_sq.append(poly_at_beta_sq)
        transcript.absorb(poly_at_beta)
        transcript.absorb(poly_at_neg_beta)

    transcript.absorb(evals_sq[0])

    print("evals_pos=", evals_pos)
    print("evals_neg=", evals_neg)
    print("evals_sq=", evals_sq)

    gamma = transcript.squeeze()
    
    if debug > 0: 
        print("gamma=", gamma)

    h_agg = [0] * n
    for i in range(n):
        for j in range(k):
            poly = h_uni_poly_vec[j]
            if i < len(poly):
                h_agg[i] += poly[i] * (gamma**j)
    if debug > 1: 
        print("h_agg=", h_agg)
    
    h_agg_poly_at_beta = uni_eval_from_coeffs(h_agg, beta)
    h_agg_poly_at_neg_beta = uni_eval_from_coeffs(h_agg, -beta)
    h_agg_poly_at_beta_sq = uni_eval_from_coeffs(h_agg, beta^2)

    if debug > 1: 
        print("h_agg_poly_at_beta={}, h_agg_poly_at_neg_beta={}, h_agg_poly_at_beta_sq={}".format(h_agg_poly_at_beta, h_agg_poly_at_neg_beta, h_agg_poly_at_beta_sq))

    # DEBUG: verify h_agg
    if debug > 0:
        print("check: aggregated h(X) must be equal to sum(h_i(X)) at (beta, -beta, beta^2)")
        assert h_agg_poly_at_beta == sum([evals_pos[i]*(gamma**i) for i in range(k)])
        assert h_agg_poly_at_neg_beta == sum([evals_neg[i]*(gamma**i) for i in range(k)])
        assert h_agg_poly_at_beta_sq == sum([evals_sq[i]*(gamma**i) for i in range(k)])

    # DEBUG: verify evaluations
    if debug > 0:
        dbg_evals_sq = evals_sq[:] + [v]
        print("dbg_evals_sq=", dbg_evals_sq)
        for i in range(k):
            print("check: h_{i+1}(r^2) = h_i(r) + h_i(-r))/2 + x_i * (h_i(r) - h_i(-r))/(2*r)")
            print("i={}, {}, {}".format(i, 
                dbg_evals_sq[i+1], interp_and_eval_line(evals_pos[i], evals_neg[i], point[i], beta)))

    h_agg_poly = UniPolynomial(h_agg)
    interp_poly = UniPolynomial.interpolate([h_agg_poly_at_beta, h_agg_poly_at_neg_beta, h_agg_poly_at_beta_sq], [beta, -beta, beta^2])
    vanishing_poly = UniPolynomial([-beta, 1])*UniPolynomial([beta, 1]) * UniPolynomial([-beta*beta, 1])
    g_poly = h_agg_poly - interp_poly
    quo_poly, rem = divmod(g_poly, vanishing_poly)

    if debug > 0:
        # assert rem == UniPolynomial([0])
        pass

    if debug > 1:
        print("interp_poly=", interp_poly)
        print("vanishing_poly=", vanishing_poly)    
        print("quo={}, rem={}".format(quo_poly, rem))
        print("quo_poly={}".format(quo_poly))
        print("rem={}".format(rem))
        print("quo={}".format(quo_poly))
    
    Cq = kzg10.commit(quo_poly.coeffs)
    transcript.absorb(Cq)

    zeta = transcript.squeeze()
    print("zeta={}".format(zeta))
    
    interp_at_zeta =interp_poly.evaluate(zeta)
    print("interp_at_zeta={}".format(interp_at_zeta))
    
    r_poly = h_agg_poly - UniPolynomial([interp_at_zeta]) - quo_poly * ((zeta - beta) * (zeta + beta) * (zeta - beta*beta))
    print("h_agg_poly - UniPolynomial([interp_at_zeta])=", h_agg_poly - UniPolynomial([interp_at_zeta]))
    print("quo_poly * ((zeta - beta) * (zeta + beta) * (zeta - beta*beta))=", quo_poly * ((zeta - beta) * (zeta + beta) * (zeta - beta*beta)))
    print("r_poly=", r_poly)

    print("r(zeta)={}".format(r_poly.evaluate(zeta)))

    Cr = kzg10.commit(r_poly.coeffs)

    transcript.absorb(Cr)

    arg_r_poly_at_zeta =kzg10.prove_eval(r_poly.coeffs, point=zeta, v=0)

    # DEBUG: verify r_poly argument
    if debug > 0:
        print("check: `r(z)=0` argument must hold")
        assert kzg10.verify_eval(Cr, arg_r_poly_at_zeta, point=zeta, v=0)

    return (h_uni_cm_vec, evals_pos, evals_neg, evals_sq[:1], Cq, arg_r_poly_at_zeta)

In [75]:
def verify_eval_arg_opt_opt(C, arg, point, v, transcript, debug=0):
    """
    Verify an optimized evaluation argument proof.

    Args:
    C: commitment to f
    arg: argument to be verified (output of prove_eval_arg_opt)
    point: evaluation point
    v: claimed evaluation value
    transcript: proof transcript
    """
    k = len(point)
    n = 1 << k
    
    if debug > 1:
        print("C={}, arg={}, point={}, v={}".format(C, arg, point, v))

    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    h_uni_cm_vec, evals_pos, evals_neg, evals_sq, Cq, Cw = arg

    for i in range(len(h_uni_cm_vec)):
        transcript.absorb(h_uni_cm_vec[i])
    
    beta = transcript.squeeze()
    if debug > 1:
        print("> beta = {}".format(beta))

    for i in range(k):
        transcript.absorb(evals_pos[i])
        transcript.absorb(evals_neg[i])
        evals_sq.append(interp_and_eval_line(evals_pos[i], evals_neg[i], point[i], beta))
    transcript.absorb(evals_sq[0])

    if debug > 1:
        print("> evals_sq = {}".format(evals_sq))
    
    # 1st check:
    if debug > 0:
        print("🛡️: check if the evaluation f({})={} is correct".format(point, v))
        print("    by computing the folded polynomial h_{}({}^2) ?= {}".format(k, beta, v))
        if (evals_sq[-1] == v): 
            print("😀 ✅")
    if not (evals_sq[-1] == v):
        return False
    
    gamma = transcript.squeeze()
    if debug > 1:
        print("> gamma = {}".format(gamma))

    Ch = C
    for i in range(0, k-1):
        Ch = Ch + (gamma**(i+1)) * h_uni_cm_vec[i]

    if debug > 1:
        print("> Commitment of h(X): Ch={}".format(Ch))
    
    # Compute h_agg(beta), h_agg(-beta), h_agg(beta^2)
    h_agg_poly_at_beta = sum([evals_pos[i]*(gamma**i) for i in range(k)])
    h_agg_poly_at_neg_beta = sum([evals_neg[i]*(gamma**i) for i in range(k)])
    h_agg_poly_at_beta_sq = sum([evals_sq[i]*(gamma**i) for i in range(k)])

    if debug > 1:
        print("> h_agg_poly_at_beta={}".format(h_agg_poly_at_beta))
        print("> h_agg_poly_at_neg_beta={}".format(h_agg_poly_at_neg_beta))
        print("> h_agg_poly_at_beta_sq={}".format(h_agg_poly_at_beta_sq))

    # Interpolate c(X) from three points: 
    #
    #    [(beta, h_agg(beta)), (-beta, h_agg(-beta)), (beta^2, h_agg(beta^2))]

    interp_poly = UniPolynomial.interpolate(
        [h_agg_poly_at_beta, h_agg_poly_at_neg_beta, h_agg_poly_at_beta_sq], 
        [beta, -beta, beta**2])

    # h_agg(X) - c(X) = q(X) * (X-beta)(X+beta)(X-beta^2)
    # 
    # Receive Cq = Commit(c(X)) 
    transcript.absorb(Cq)

    # Sample randomness
    zeta = transcript.squeeze()

    if debug > 1:
        print("> zeta = {}".format(zeta))
    
    # vanishing_poly_at_zeta= vanishing_poly.evaluate(zeta)
    interp_at_zeta = interp_poly.evaluate(zeta)
    vanishing_poly_at_zeta = ((zeta - beta) * (zeta + beta) * (zeta - beta*beta))

    print("interp_at_zeta={}".format(interp_at_zeta))

    Cr = Ch - kzg10.commit([interp_at_zeta]) - Cq * vanishing_poly_at_zeta

    if debug > 1:
        print("> Cr={}".format(Cr))
        print("> Cq={}".format(Cq))

    transcript.absorb(Cr)

    # Verify r_poly argument
    
    # 2. Check if r(z)=0
    Cw_verified = kzg10.verify_eval(Cr, Cw, point=zeta, v=0)

    if debug > 0:
        if Cw_verified: print("🛡️: check if r(z)=0 😀 ✅")
        else: print("🛡️: check if r(z)=0 😢 ❌")

    if not Cw_verified:
        return False

    return True

In [76]:
arg = prove_eval_arg_opt_opt(ff_cm, [1, 3, 2, 1], 
            point=[91, 181], v=123, transcript=transcript.clone(), debug=1); arg

check: fully folded polynomial must be equal to the claimed evaluation: 123 = 123
coeffs= [123]
h_uni_cm_vec= [Commitment(93*X0 + 81)]
h_uni_poly_vec= [[1, 3, 2, 1], [81, 93]]
beta= 117
evals_pos= [37, 154]
evals_neg= [102, 8]
evals_sq= [116, 130]
gamma= 103
check: aggregated h(X) must be equal to sum(h_i(X)) at (beta, -beta, beta^2)
dbg_evals_sq= [116, 130, 123]
check: h_{i+1}(r^2) = h_i(r) + h_i(-r))/2 + x_i * (h_i(r) - h_i(-r))/(2*r)
i=0, 130, 130
check: h_{i+1}(r^2) = h_i(r) + h_i(-r))/2 + x_i * (h_i(r) - h_i(-r))/(2*r)
i=1, 123, 123
zeta=159
interp_at_zeta=152
h_agg_poly - UniPolynomial([interp_at_zeta])= 86 + 125x + 2x^2 + x^3
quo_poly * ((zeta - beta) * (zeta + beta) * (zeta - beta*beta))= 146
r_poly= 133 + 125x + 2x^2 + x^3
r(zeta)=0
check: `r(z)=0` argument must hold


([Commitment(93*X0 + 81)],
 [37, 154],
 [102, 8],
 [116],
 Commitment(1),
 Commitment(X0^2 - 32*X0 + 55))

In [77]:
verify_eval_arg_opt_opt(ff_cm, arg, point=[91, 181], v=123, transcript=transcript.clone(), debug=1)

🛡️: check if the evaluation f([91, 181])=123 is correct
    by computing the folded polynomial h_2(117^2) ?= 123
😀 ✅
interp_at_zeta=152
🛡️: check if r(z)=0 😀 ✅


True